## Overview of Self-Attention
---
The purpose of attention is to consider each input's relation to all other inputs in a sequence. For the case of NLP, this would mean words. For example, in the sentence "This meal was so good I would eat it every day." the ideal attention would specify high values between "meal", "good", and "it" (+ maybe "eat"). 

For the case of vision transformers, the sequential inputs are uniformly divided MxM patches of the input image. So input 1 might be the top left corner, input 2 the top middle, input 3 top right, input 4 middle left, and so on. This way, attention picks up on the relations between areas of an image. When put into transformers, this makes for efficient representation of the entire image, rather than relying on convolutions that focus on isolated regions.

## The Details
---
1. The inputs are first linearly projected into three representations: the *query*, the *key*, and the *value*


2. The query and the key are used to learn attention through their dot product. For each position X in the query and position Y in the key, the attention score XY represents the importance of the input at position X in describing the input at position Y.


3. The attention scores are then passed through a softmax layer so all scores are form 0 to 1.


4. Each attention score XY is then multiplied by the value representation at position X and all scores describing position Y are summed. This final value is the new embedding at position Y.


![](https://miro.medium.com/max/1400/1*AVz57aVsuQniiOtwABx5KA.png)

## PyTorch Implementation
---

In [ ]:
from torch import nn

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x